In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
os.chdir('drive/My Drive/tensorflow_data')

In [0]:
!ls -lht

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
batch_size = 64
n_batch = mnist.train.num_examples // batch_size
print(mnist.train.num_examples, n_batch)    # 55000 859

W0825 05:00:29.534807 140348387104640 deprecation.py:323] From <ipython-input-5-6b146b2591c9>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0825 05:00:29.537243 140348387104640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0825 05:00:29.541200 140348387104640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0825 05:00:31.259292 140348387104640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.


Extracting MNIST_data/train-labels-idx1-ubyte.gz


W0825 05:00:31.803504 140348387104640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W0825 05:00:32.736302 140348387104640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


55000 859


In [6]:
# 输入图片是28*28
n_inputs = 28    # 输入一行（按行输入），一行有28个数据
max_time = 28    # max_time是这批数据中最长序列的长度
lstm_size = 1000    # 隐层单元，是一个超参数
n_classes = 10    # 10个分类

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

## 【注意：LSTM只需要输入输出层的参数W_ya即可，其他的参数会自己随机初始化】 ##
weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[n_classes]))

# 定义RNN网络
def RNN(x, weights, bias):
    inputs = tf.reshape(x, [-1, max_time, n_inputs])
    # 定义LSTM
    lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_size)
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    ## outputs是一个tensor，如果time_major=True, 则outputs的形状为：[max_time, batch_size, cell.output_size]
    ##                      如果time_major=False, 则outputs的形状为：[batch_size, max_time, cell.output_size] 默认为False
    ## final_state=[state, batch_size, cell.state_size]
    ## final_state[0]是cell state
    ## final_state[1]是hidden_state
    results = tf.nn.softmax(tf.matmul(final_state[1], weights)+bias)
    return results

pred = RNN(x, weights, bias)

loss = tf.losses.softmax_cross_entropy(y, pred)
train = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_pred = tf.equal(tf.argmax(y,1), tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(31):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={x:batch_xs, y:batch_ys})
            # print('{}:{}'.format(epoch, batch), end=' ')
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('Epoch {}, test accuracy is {}'.format(epoch, acc))

W0825 05:02:30.550001 140348387104640 deprecation.py:323] From <ipython-input-6-7337e37605fc>:17: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0825 05:02:30.551370 140348387104640 deprecation.py:323] From <ipython-input-6-7337e37605fc>:18: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0825 05:02:30.625717 140348387104640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead

Epoch 0, test accuracy is 0.9212999939918518
Epoch 1, test accuracy is 0.949999988079071
Epoch 2, test accuracy is 0.9593999981880188
Epoch 3, test accuracy is 0.9660999774932861
Epoch 4, test accuracy is 0.9699000120162964
Epoch 5, test accuracy is 0.972599983215332
Epoch 6, test accuracy is 0.9696999788284302
Epoch 7, test accuracy is 0.9804999828338623
Epoch 8, test accuracy is 0.9789000153541565
Epoch 9, test accuracy is 0.9837999939918518
Epoch 10, test accuracy is 0.9814000129699707
Epoch 11, test accuracy is 0.977400004863739
Epoch 12, test accuracy is 0.9861000180244446
Epoch 13, test accuracy is 0.9782999753952026
Epoch 14, test accuracy is 0.9882000088691711
Epoch 15, test accuracy is 0.9832000136375427
Epoch 16, test accuracy is 0.986299991607666
Epoch 17, test accuracy is 0.9886999726295471
Epoch 18, test accuracy is 0.9858999848365784
Epoch 19, test accuracy is 0.9869999885559082
Epoch 20, test accuracy is 0.9855999946594238
Epoch 21, test accuracy is 0.987500011920929
Epo

与07中用CNN对mnist进行分类对比发现，LSTM循环30个周期的效果仍然赶不上CNN循环20个周期的效果，所以可以看出CNN在处理图像数据时确实有优势。

In [7]:
!/opt/bin/nvidia-smi

Sun Aug 25 05:19:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |   1313MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  